In [82]:
import pandas as pd
import numpy as np
import sklearn

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
wordnet_lemmatizer = WordNetLemmatizer()
ps = PorterStemmer()

import string
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()
from pprint import pprint
from unidecode import unidecode
import random
import pandas as pd
import re
import string
from nltk.stem.snowball import SnowballStemmer
import os
import gensim
from gensim.models import Doc2Vec
from nltk.corpus import stopwords
from sklearn.cluster import KMeans
from sklearn import metrics
import matplotlib.pyplot as plt
import pickle
import re
import os
import gensim
from sklearn.decomposition import PCA

import warnings
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
warnings.filterwarnings("ignore")

nltk.download('stopwords')
nltk.download('punkt')

pd.set_option('display.max_colwidth', None)

[nltk_data] Downloading package stopwords to /Users/gigi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/gigi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [98]:
question_pairs.sample()

,context,question,cleaned_lowercase_nostop
3243,"the prime minister has the right to appoint a maximum of three such ministers, as the limit of ministers in one government is fifteen. it is also known as the cabinet. the cabinet carries out the country's domestic and foreign policy, shaped by parliament; it directs and co-ordinates the work of government institutions and bears full responsibility for everything occurring within the authority of executive power. the government, headed by the prime minister, thus represents the political leadership of the country and makes decisions in the name of the whole executive power.",What is another name for the governing body of ministers?,prime minister right appoint maximum three ministers limit minister one government fifteen also known cabinet cabinet carry country s domestic foreign policy shaped parliament directs co ordinate work government institution bear full responsibility everything occurring within authority executive power government headed prime minister thus represents political leadership country make decision name whole executive power


In [28]:
question_pairs = pd.read_csv('legal_squad_data.csv', encoding='utf-8').loc[:, ['context', 'question']]
question_pairs = question_pairs.drop_duplicates(subset=['context', 'question'])

,context,question
0,"In 1785, the assembly of the Congress of the C...",In what year did New York become the United St...
1,"In 1785, the assembly of the Congress of the C...",Who was the United States' first President?
2,"In 1785, the assembly of the Congress of the C...",In what building did the Supreme Court of the ...
3,"In 1785, the assembly of the Congress of the C...",On what street did the writing of the Bill of ...
4,"In 1785, the assembly of the Congress of the C...",What was the second largest city in the United...


In [29]:
stop_words = stopwords.words("english")

def clean_lowercase_nostop(text):
    text = ' '.join([wordnet_lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words])
#     text = ' '.join([ps.stem(word) for word in text.split() if word not in stop_words])
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\s{2,}', " ", text)
    text = unidecode(text)
    return text.lower()

In [30]:
question_pairs['context'] = question_pairs['context'].apply(lambda x: str(x).lower())
question_pairs['cleaned_lowercase_nostop'] = question_pairs['context'].progress_apply(clean_lowercase_nostop)

In [31]:
corpus = question_pairs['cleaned_lowercase_nostop'].tolist()
corpus = list(np.unique(corpus))
corpus[:10]

[' new labour first termed alternative branding labour party dating conference slogan first used labour party 1994 later seen draft manifesto published party 1996 called new labour new life britain continuation trend begun leadership neil kinnock new labour name official status remains common use distinguish modernisers holding traditional positions normally referred old labour ',
 ' note 5 north carolina constitution 1776 disestablished anglican church 1835 nc constitution allowed protestant hold public office 1835 1876 allowed christian including catholics hold public office article vi section 8 current nc constitution forbids atheist holding public office clause held united state supreme court unenforceable 1961 case torcaso v watkins court ruled unanimously clause constituted religious test incompatible first fourteenth amendment protections ',
 ' religious test clause interpreted cover elected official appointed ones career civil servant well political appointees religious belief 

In [36]:
import spacy
import en_core_web_sm

nlp = en_core_web_sm.load()

In [37]:
def most_similar(word, topn=5):
    word = nlp.vocab[str(word)]
    queries = [
      w for w in word.vocab 
      if w.is_lower == word.is_lower and w.prob >= -15 and np.count_nonzero(w.vector)
    ]

    by_similarity = sorted(queries, key=lambda w: word.similarity(w), reverse=True)
    return [(w.lower_,w.similarity(word)) for w in by_similarity[:topn+1] if w.lower_ != word.lower_]

Sentence (or document) objects  have vectors, derived from the averages of individual token vectors. This makes it possible to compare similarities between whole documents.

In [38]:
doc = nlp('The quick brown fox jumped over the lazy dogs.')
len(doc.vector)

96

### Bert Sentence Transformer

In [39]:
from sentence_transformers import SentenceTransformer
import scipy.spatial
embedder = SentenceTransformer('bert-base-nli-mean-tokens')

In [40]:
%%time
corpus_embeddings = embedder.encode(corpus)

CPU times: user 4min 34s, sys: 12.6 s, total: 4min 47s
Wall time: 2min 39s


----
## Candidate Genration using Faiss vector similarity search library

Faiss is a library developed by Facebook AI Research. It is for effecient similarity search and clustering of dense vectors.

**References:**

1. [Tutorial](https://github.com/facebookresearch/faiss/wiki/Getting-started)
2. [facebookresearch/faiss](https://github.com/facebookresearch/faiss)

In [65]:
import faiss
d = 768
index = faiss.IndexFlatL2(d)
print(index.is_trained)
index.add(np.stack(corpus_embeddings, axis=0))
print(index.is_trained)
print(index.ntotal)

True
True
1080


In [99]:
# queries = ['What is the step by step guide to invest in share market in india?', 'How can Internet speed be increased by hacking through DNS?']
orig_query = "What is another name for the governing body of ministers?"
query = orig_query.lower()
query = clean_lowercase_nostop(query)
queries = [query]
query_embeddings = embedder.encode(queries)

In [77]:
k = 5
D, I = index.search(np.stack(query_embeddings, axis=0), k)
print(I)

[[ 600   17  480 1030  497]]


In [100]:
print("Orignal Query:", orig_query)
for query, query_embedding in zip(queries, query_embeddings):
    distances, indices = index.search(np.asarray(query_embedding).reshape(1,768),k)
    print("\n======================\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")
    for idx in range(0,5):
        print(corpus[indices[0,idx]], "(Distance: %.4f)" % distances[0,idx])
        print()

Orignal Query: What is another name for the governing body of ministers?


Query: another name governing body ministers 

Top 5 most similar sentences in corpus:
council minister - presidency prime minister or president portugal latter s request minister may also include one deputy prime ministers - act cabinet government required define broad outline policy programme present assembly mandatory period debate failure assembly reject government programme absolute majority deputy confirms cabinet office  (Distance: 193.0990)

signature parties representative follow end text treaty later reprinted collection treaty currently effect editor often append date respective party ratified treaty came effect party  (Distance: 204.3268)

well head government prime minister may role titles--the prime minister united kingdom example also first lord treasury minister civil service prime minister may take ministerial posts--for example second world war winston churchill also minister defence although m

In [ ]:
### did not use the code from here onwards

----
## Reranking using Bidirectional LSTM model

<img src="img/bi_lstm.png" alt="Drawing" style="width: 600px;"/>

**Reference:** https://mlwhiz.com/blog/2019/03/09/deeplearning_architectures_text_classification/

In [48]:
import re
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import WordNetLemmatizer, SnowballStemmer
toko_tokenizer = ToktokTokenizer()
wordnet_lemmatizer = WordNetLemmatizer()

def normalize_text(text):
        puncts = ['/', ',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
         '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
         '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
         '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
         '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

        def clean_text(text):
            text = str(text)
            text = text.replace('\n', '')
            text = text.replace('\r', '')
            for punct in puncts:
                if punct in text:
                    text = text.replace(punct, '')
            return text.lower()

        def clean_numbers(text):
            if bool(re.search(r'\d', text)):
                text = re.sub('[0-9]{5,}', '#####', text)
                text = re.sub('[0-9]{4}', '####', text)
                text = re.sub('[0-9]{3}', '###', text)
                text = re.sub('[0-9]{2}', '##', text)
            return text

        contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}

        def _get_contractions(contraction_dict):
            contraction_re = re.compile('(%s)' % '|'.join(contraction_dict.keys()))
            return contraction_dict, contraction_re

        contractions, contractions_re = _get_contractions(contraction_dict)

        def replace_contractions(text):
            def replace(match):
                return contractions[match.group(0)]
            return contractions_re.sub(replace, text)

        stopword_list = nltk.corpus.stopwords.words('english')

        def remove_stopwords(text, is_lower_case=True):
            tokens = toko_tokenizer.tokenize(text)
            tokens = [token.strip() for token in tokens]
            if is_lower_case:
                filtered_tokens = [token for token in tokens if token not in stopword_list]
            else:
                filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
            filtered_text = ' '.join(filtered_tokens)    
            return filtered_text

        def lemmatizer(text):
            tokens = toko_tokenizer.tokenize(text)
            tokens = [token.strip() for token in tokens]
            tokens = [wordnet_lemmatizer.lemmatize(token) for token in tokens]
            return ' '.join(tokens)

        def trim_text(text):
            tokens = toko_tokenizer.tokenize(text)
            tokens = [token.strip() for token in tokens]
            return ' '.join(tokens)
        
        def remove_non_english(text):
            tokens = toko_tokenizer.tokenize(text)
            tokens = [token.strip() for token in tokens]
            tokens = [token for token in tokens if d.check(token)]
            eng_text = ' '.join(tokens)
            return eng_text

        text_norm = clean_text(text)
        text_norm = clean_numbers(text_norm)
        text_norm = replace_contractions(text_norm)
#         text_norm = remove_stopwords(text_norm)
#         text_norm = remove_non_english(text_norm)
        text_norm = lemmatizer(text_norm)
        text_norm = trim_text(text_norm)
        return text_norm

In [328]:
question_pairs.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [329]:
question_pairs.shape

(404351, 6)

In [293]:
embedding_path = "./../../Embeddings/glove.twitter.27B/glove.twitter.27B.200d.txt"
def get_word2vec(file_path):
    file = open(embedding_path, "r")
    if (file):
        word2vec = dict()
        split = file.read().splitlines()
        for line in split:
            key = line.split(' ',1)[0]
            value = np.array([float(val) for val in line.split(' ')[1:]])
            word2vec[key] = value
        return (word2vec)
    else:
        print("invalid fiel path")
w2v = get_word2vec(embedding_path)

In [352]:
total_text = pd.concat([question_pairs['question1'], question_pairs['question2']]).reset_index(drop=True)
total_text = total_text.apply(lambda x: str(x))
total_text = total_text.apply(lambda x: normalize_text(x))
max_features = 6000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(total_text)
question_1_sequenced = tokenizer.texts_to_sequences(question_pairs['question1'].apply(lambda x: normalize_text(x)))
question_2_sequenced = tokenizer.texts_to_sequences(question_pairs['question2'].apply(lambda x: normalize_text(x)))
vocab_size = len(tokenizer.word_index) + 1

In [353]:
vocab_size

92423

In [354]:
maxlen = 100
question_1_padded = pad_sequences(question_1_sequenced, maxlen=maxlen)
question_2_padded = pad_sequences(question_2_sequenced, maxlen=maxlen)

In [359]:
y = question_pairs['is_duplicate']

In [381]:
from tqdm import tqdm

In [384]:
from numpy import zeros
embedding_matrix = zeros((vocab_size, 768))
for word, i in tokenizer.word_index.items():
    embedding_vector = w2v.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector[0]

In [357]:
embedding_size = 128
max_len = 100

inp1 = Input(shape=(100,))
inp2 = Input(shape=(100,))

x1 = Embedding(vocab_size, 200, weights=[embedding_matrix], input_length=max_len)(inp1)
x2 = Embedding(vocab_size, 200, weights=[embedding_matrix], input_length=max_len)(inp2)

x3 = Bidirectional(LSTM(32, return_sequences = True))(x1)
x4 = Bidirectional(LSTM(32, return_sequences = True))(x2)

x5 = GlobalMaxPool1D()(x3)
x6 = GlobalMaxPool1D()(x4)

x7 =  dot([x5, x6], axes=1)

x8 = Dense(40, activation='relu')(x7)
x9 = Dropout(0.05)(x8)
x10 = Dense(10, activation='relu')(x9)
output = Dense(1, activation="sigmoid")(x10)

model = Model(inputs=[inp1, inp2], outputs=output)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
batch_size = 256
epochs = 4

In [360]:
model.fit([question_1_padded, question_2_padded], y, batch_size=batch_size, epochs=epochs, validation_split=0.2, )

Train on 323480 samples, validate on 80871 samples
Epoch 1/4
323480/323480 [==============================] - 1096s 3ms/step - loss: 0.5212 - acc: 0.7372 - val_loss: 0.4596 - val_acc: 0.7802
Epoch 2/4
323480/323480 [==============================] - 1101s 3ms/step - loss: 0.4169 - acc: 0.8038 - val_loss: 0.4300 - val_acc: 0.7978
Epoch 3/4
323480/323480 [==============================] - 1135s 4ms/step - loss: 0.3536 - acc: 0.8400 - val_loss: 0.4340 - val_acc: 0.7976
Epoch 4/4
323480/323480 [==============================] - 1077s 3ms/step - loss: 0.2963 - acc: 0.8701 - val_loss: 0.4411 - val_acc: 0.8035


--------
## Combining candidate generation and reranking

In [361]:
query

'How will Indian GDP be affected from banning 500 and 1000 rupees notes?'

In [363]:
query_copy = [query]*len(relevant_docs)
question_1_sequenced_final = tokenizer.texts_to_sequences(query_copy)
question_2_sequenced_final = tokenizer.texts_to_sequences(relevant_docs)

In [364]:
maxlen = 100
question_1_padded_final = pad_sequences(question_1_sequenced_final, maxlen=maxlen)
question_2_padded_final = pad_sequences(question_2_sequenced_final, maxlen=maxlen)

In [365]:
preds_test = model.predict([question_1_padded_final, question_2_padded_final])
preds_test = np.array([x[0] for x in preds_test])

In [390]:
[relevant_docs[x] for x in preds_test.argsort()[::-1]][:10]

['What do you think about banning 500 and 1000 rupee notes in India?',
 'What will be the implications of banning 500 and 1000 rupees currency notes on Indian economy?',
 'What will be the consequences of 500 and 1000 rupee notes banning?',
 'What will be the effects after banning on 500 and 1000 rupee notes?',
 'What will be the impact on real estate by banning 500 and 1000 rupee notes from India?',
 'How is banning 500 and 1000 INR going to help Indian economy?',
 'What are your views on India banning 500 and 1000 notes? In what way it will affect Indian economy?',
 'How is discontinuing 500 and 1000 rupee note going to put a hold on black money in India?',
 'What will be the result of banning 500 and 1000 rupees note in India?',
 'What are the economic implications of banning 500 and 1000 rupee notes?']